# Transposition DataFrame, SQL

Hackathon Leipzig 2023

In [ ]:
import pandas as pd
import sqlalchemy

URLs der CVS-Datensätze

In [ ]:
urlOepnv = 'https://statistik.leipzig.de/opendata/api/values?kategorie_nr=10&rubrik_nr=4&periode=y&format=csv'
urlGeburten = 'https://opendata.leipzig.de/dataset/9a96c299-ab95-4468-ae75-f207eb1938a4/resource/fce95310-77f0-40e4-9906-f5fee95aea3f/download/geburtenundfertilitat.csv'

Einlesen in Pandas-Dataframe

In [ ]:
oepnv = pd.read_csv(urlOepnv, decimal=',')
oepnv

Umbenennen der Features, da doppelte Spaltennamen

In [ ]:
features = ['tr_lines', 'tr_len', 'tr_vehicles', 'tr_km', 'tr_persons',
            'bu_lines', 'bu_len', 'bu_vehicles', 'bu_km', 'bu_persons',]
oepnv['Kennziffer'] = features
oepnv

Einlesen des zweiten Dataframes

In [ ]:
geburten = pd.read_csv(urlGeburten)
geburten

Bereinigen des ersten Dataframes. Spalte Einheit enthält keine sinnvollen Daten.

In [ ]:
oepnv0 = oepnv.drop('Einheit', axis=1)
oepnv0

## Variante 1

Transponieren der NumPy-Matrix

- Auswahl der notwendigen Spalten
- Umwandeln in NumPy-Array
- Transponieren
- Umwandeln in DataFrame
- Spaltennamen setzen
- Jahresspalte hinzufügen

In [ ]:
#data = oepnv0.drop('Kennziffer', axis=1).values
data = oepnv0.iloc[:, 1:].values
data = data.T
oepnvt = pd.DataFrame(data, columns=features)
oepnvt['year'] = oepnv0.columns[1:]
oepnvt

## Variante 2: stack() und unstack()

In [ ]:
oepnv0

Spalte, der die folgenden Spaltenwerte zeilenweise zugeordnet werden, zum Index machen.

In [ ]:
oepnv1 = oepnv0.set_index('Kennziffer')
oepnv1

Spalten stapeln

In [ ]:
oepnv2 = oepnv1.stack()
oepnv2.head(30)

Index zurücksetzen und die Spalten umbenennen

In [ ]:
oepnv3 = oepnv2.reset_index()
oepnv3.columns = ['feature', 'year', 'value']
oepnv3

Neuen Index setzen:

- erster Spalte: bleibt Spalte
- zweite Spalte: Index der neuen Spalten

In [ ]:
oepnv4 = oepnv3.set_index(['year', 'feature'])
oepnv4

Zeilen zu Spalten

In [ ]:
oepnvt2tmp = oepnv4.unstack()
oepnvt2tmp

Zeilenindex zur Jahresspalte machen, Spaltenüberschriften neu setzen

In [ ]:
oepnvt2 = oepnvt2tmp.reset_index()
oepnvt2.columns = ['year',] + features
oepnvt2

## SQL-Operationen

Engine für jeweilige DB erzeugen.

In [ ]:
engine = sqlalchemy.create_engine('sqlite+pysqlite:///hackathon2023.sqlite3')

Schreiben der Tabelle, Index wird verworfen, Überschreiben, falls vorhanden

In [ ]:
oepnvt2.to_sql('oepnv', engine, if_exists='replace', index=False)

Zweite Tabelle analog.

In [ ]:
geburten.to_sql('geburten', engine, if_exists='replace', index=False)

Einlesen einer Einzeltabelle aus der Datenbank

In [ ]:
x = pd.read_sql_table('geburten', engine)
x

Einlesen einer SQL-Abfrage in einen Dataframe

In [ ]:
x = pd.read_sql('select a.*, b.* from oepnv as a left join geburten as b on a.year=b.Jahr', engine)
x